# UKF short example 
From: https://drive.google.com/drive/folders/1zHVEtsGyXWVNFN3eHixqFYO1oOlAyiZ1


<table><tr>
<td> <img src="data/uk1.png" alt="EKF short example 1" width="680"/> </td>
<td> <img src="data/uk2.png" alt="EKF short example 2" width="680"/> </td>
</tr></table>


<table><tr>
<td> <img src="data/uk3.png" alt="EKF short example 3" width="680"/> </td>
<td> <img src="data/uk4.png" alt="EKF short example 4" width="680"/> </td>
</tr></table>

In [57]:
import numpy as np 
from numpy.linalg import inv, cholesky

dt = 0.5
N = 2
K= 3 - N
u0 = -2
S = 20
D = 40
y1 = np.pi/6
R = np.array([[0.01]])

F = np.identity(2)
F[0, 1] = dt
G = np.zeros((2, 1))
G[1, 0] = dt
Q0 = 0.1* np.identity(2)

P0_hat = np.diag([0.01, 1])

###################################################################
# 1 Prediction STEP
###################################################################
# (1) Compute sigma points
L = cholesky(P0_hat)
X0hat_arr = np.zeros((2*N+1, N, 1))
X0hat_arr[0] = np.array([[0, 5]]).T
for i in range(2):
    X0hat_arr[i+1] = X0hat_arr[0] + np.sqrt(N+K)*L[:, i].reshape((N, 1))
    X0hat_arr[N + i+1] = X0hat_arr[0] - np.sqrt(N+K)*L[:, i].reshape((N, 1))

# (2) Propagate sigma points
def f(x, u):
    return F.dot(x) + G.dot(u)
X1check_arr = np.zeros((2*N+1, N, 1))
for i in range(2*N+1):
    X1check_arr[i] = f(X0hat_arr[i], u0)

# (3) Compute predicted mean and covariance 
def alpha(i):
    if i == 0:
        return K / (N+K)
    return 0.5 * (1/(N+K))

X1_check = np.zeros((N, 1))
for i in range(2*N+1):
    X1_check = X1_check + alpha(i) * X1check_arr[i]

P1_check = np.zeros((N, N))
for i in range(2*N+1):
    P1_check = P1_check + alpha(i) * np.dot(X1check_arr[i] - X1_check, (X1check_arr[i] - X1_check).T)

P1_check = P1_check + Q0

###################################################################
# 2 Correction STEP
###################################################################
def h(x, S, D):
    p = x[0, 0]
    return np.arctan2(S, D - p)

# (1) Predict measurements from propagated sigma points
y1hat_arr = np.zeros((2*N+1, 1, 1))

# (2) Estimate the mean and covariance of the predicted measurements
y1hat = 0
for i in range(2*N+1):
    y1hat = y1hat + alpha(i) * y1hat_arr[i]

Py = 0
for i in range(2*N+1):
    Py += alpha(i) * np.dot(y1hat_arr[i] - y1hat, (y1hat_arr[i] - y1hat).T)
Py = Py + R

# (3) Compute cross-covariance and Kalman gain
Pxy = np.zeros( (2, 1) )
for i in range(2*N+1):
   Pxy += alpha(i) * np.dot(X1check_arr[i] - X1_check, (y1hat_arr[i] - y1hat).T )

Kk = Pxy.dot( inv(Py) )

# Compute corrected mean and covariance
X1_hat = X1_check + Kk.dot(y1 - y1hat)
print("X1_hat", X1_hat)

P1_hat = P1_check - Kk.dot(Py).dot(Kk.T)
print("P1_hat", P1_hat)

X1_hat [[2.5]
 [4. ]]
P1_hat [[0.36 0.5 ]
 [0.5  1.1 ]]
